In [ ]:
#director, lead actors, writers, producers, genre, budget, film length, revenue

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://editorial.rottentomatoes.com/guide/oscars-best-and-worst-best-pictures/"
r = requests.get(url)

In [3]:
text = r.text
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
div_tags = soup.find_all('div', {'class': 'article_movie_title'})

In [5]:
titles = []
for div in div_tags:
    a_tags = div.find_all('a')
    for a in a_tags:
        titles.append(a.get_text().strip())

In [6]:
year_tag = soup.find_all("span", {"class":"subtle start-year"})

years = []
for entry in year_tag:
    year = entry.get_text().strip()
    year = year.replace("(","").replace(")","")
    year = int(year)
    years.append(year)

In [7]:
director_tags = soup.find_all('div', {'class': 'info director'})

In [8]:
directors = []
for d in director_tags:
    directors.append(d.get_text().strip().replace('Directed By:', '').strip())

In [9]:
# actor_tags = soup.find_all('div', {'class': 'info cast'})

In [10]:
# names = []
# for actor in actor_tags:
#     a_tags = a.find_all('a')
#     for a in a_tags:
#         names.append(a.get_text().strip())
    
    # name = a.get_text().strip().replace('Starring:', '').strip()
    # names.append(name)
    # for n in names:
    #     print(n)

In [11]:
# actors = []
# for a in actor_tags:
#     a_tags = a.find_all('a')
#     for b in a_tags:
#         actors.append(b.get_text().strip())

In [12]:
# actors

In [20]:
links = []
for div in div_tags:
    link = div.find("a")["href"]
    links.append(link)

In [21]:
producers = []
distributors = []

for link in links:
    r_sub = requests.get(link)
    soup_sub = BeautifulSoup(r_sub.text)
    try:
        distributor = soup_sub.find("rt-text", {"data-qa": "item-value"}).get_text().strip()
        distributors.append(distributor)
    except:     
        distributors.append("None")
    # try:
    #     producer = soup_sub.find("rt-link", {"href": "/celebrity/"}).get_text().strip()
    #     producers.append(producer)
        
    # except:
    #     producers.append("None")

In [18]:
with open('omdb_api.txt', 'r') as file:
    apikey = file.read()

In [59]:
parameters = {'apikey': apikey, 
             't': 'parasite'}

In [60]:
response = requests.get('http://www.omdbapi.com', params=parameters)
print(response.ok)
print(response.url)

True
http://www.omdbapi.com/?apikey=6e062c76&t=parasite
200


In [70]:
film_info = response.json()
film_info['Runtime']

'132 min'

In [90]:
runtimes = []
genres = []
writers = []
revenue = []
actors = []

for title in titles:
    parameters = {'apikey': apikey, 
             't': title}
    
    response = requests.get('http://www.omdbapi.com', params=parameters)

    film_info = response.json()
    length = film_info['Runtime']
    runtimes.append(length)
    genre = film_info['Genre']
    genres.append(genre)
    writer = film_info['Writer']
    writers.append(writer)
    box_office = film_info['BoxOffice']
    revenue.append(box_office)
    actor = film_info['Actors']
    actors.append(actors)

In [97]:
film_info['Actors']

'Bessie Love, Anita Page, Charles King'

In [99]:
film_data = pd.DataFrame({'Movie Title': titles,
              'Year Released': years,
              'Director': directors,
              "Distributor": distributors,
              "Runtime": runtimes,
              "Genre": genres,
              "Writer(s)": writers,
              "Box Office": revenue})
film_data

,Movie Title,Year Released,Director,Distributor,Runtime,Genre,Writer(s),Box Office
0,Parasite,2019,Bong Joon Ho,NEON,132 min,"Drama, Thriller","Bong Joon Ho, Han Jin-won","$53,369,749"
1,Casablanca,1942,Michael Curtiz,Warner Bros. Pictures,102 min,"Drama, Romance, War","Julius J. Epstein, Howard Koch, Murray Burnett","$4,219,709"
2,On the Waterfront,1954,Elia Kazan,Columbia Pictures,108 min,"Crime, Drama, Thriller","Budd Schulberg, Malcolm Johnson, Robert Siodmak",N/A
3,All About Eve,1950,Joseph L. Mankiewicz,20th Century Fox,138 min,Drama,"Joseph L. Mankiewicz, Mary Orr","$63,463"
4,Moonlight,2016,Barry Jenkins,A24,111 min,Drama,"Barry Jenkins, Tarell Alvin McCraney","$27,854,932"
...,...,...,...,...,...,...,...,...
93,Cavalcade,1933,Frank Lloyd,Fox,112 min,"Drama, Romance, War","Noël Coward, Reginald Berkeley",N/A
94,Out of Africa,1985,Sydney Pollack,MCA/Universal Pictures [us],161 min,"Biography, Drama, Romance","Karen Blixen, Judith Thurman, Errol Trzebinski","$87,071,205"
95,Cimarron,1931,Wesley Ruggles,RKO Radio Pictures,123 min,"Drama, Western","Edna Ferber, Howard Estabrook, Louis Sarecky",N/A
96,The Greatest Show on Earth,1952,Cecil B. DeMille,Paramount Pictures,152 min,"Drama, Family, Romance","Fredric M. Frank, Barré Lyndon, Theodore St. John","$36,000,000"
